In [2]:
import pandas as pd

df = pd.read_csv('../LCK_EventsData/matchinfo.csv')
urls = list(df['url_timeline'])

test = urls[0:2]
test

['https://gol.gg/game/stats/65431/page-timeline/',
 'https://gol.gg/game/stats/65432/page-timeline/']

In [4]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

def get_timeline_data(timeline_url):
    """
    개별 타임라인 페이지에서 events 테이블 데이터를 가져오는 함수
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    try:
        response = requests.get(timeline_url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # 타임라인 테이블 찾기
        timeline_table = soup.find('table', class_='nostyle timeline trhover')
        
        if not timeline_table:
            print(f"타임라인 테이블을 찾을 수 없습니다: {timeline_url}")
            return []
        
        # 모든 행 가져오기 (thead 제외)
        rows = timeline_table.find('tbody').find_all('tr') if timeline_table.find('tbody') else timeline_table.find_all('tr')
        
        events = []
        
        for row in rows:
            cols = row.find_all('td')
            
            # 최소 7개 이상의 열이 있는지 확인
            if len(cols) >= 7:
                # time (1열, 인덱스 0)
                time_value = cols[0].text.strip()
                
                # player (3열, 인덱스 2)
                player = cols[2].text.strip()
                
                # champion (4열, 인덱스 3) - 이미지 파일명 추출 (여러 개일 경우 연결)
                champ_col = cols[3]
                champ_imgs = champ_col.find_all('img')
                if champ_imgs:
                    # 여러 이미지의 파일명을 추출하여 ' + '로 연결
                    champ_list = []
                    for img in champ_imgs:
                        if img.get('src'):
                            filename = img['src'].split('/')[-1]
                            champ_list.append(filename)
                    champion = ' + '.join(champ_list) if champ_list else champ_col.text.strip()
                else:
                    champion = champ_col.text.strip()                
                
                # action (5열, 인덱스 4) - 이미지 파일명 추출 (여러 개일 경우 연결)
                action_col = cols[4]
                action_imgs = action_col.find_all('img')
                
                if action_imgs:
                    # 여러 이미지의 파일명을 추출하여 ' + '로 연결
                    action_list = []
                    for img in action_imgs:
                        if img.get('src'):
                            filename = img['src'].split('/')[-1]
                            action_list.append(filename)
                    action = ' + '.join(action_list) if action_list else action_col.text.strip()
                else:
                    action = action_col.text.strip()
                
                # target (7열, 인덱스 6)
                target = cols[6].text.strip()
                
                events.append({
                    'url': timeline_url,
                    'time': time_value,
                    'player': player,
                    'champion': champion,
                    'action': action,
                    'target': target
                })
        
        return events
    
    except Exception as e:
        print(f"Error fetching {timeline_url}: {str(e)}")
        return []

def scrape_timeline_data(timeline_urls):
    """
    여러 타임라인 URL에서 데이터를 수집하는 메인 함수
    """
    all_events = []
    
    for i, url in enumerate(timeline_urls, 1):
        print(f"타임라인 {i}/{len(timeline_urls)} 처리 중: {url}")
        
        events = get_timeline_data(url)
        if events:
            all_events.extend(events)
            print(f"  -> {len(events)}개의 이벤트 수집")
        
        # 서버 부하 방지를 위한 딜레이
        time.sleep(1)
    
    # 결과를 DataFrame으로 변환
    df = pd.DataFrame(all_events)
    
    # CSV로 저장
    output_file = 'timeline_events.csv'
    df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"\n데이터가 {output_file}에 저장되었습니다.")
    print(f"총 {len(df)}개의 이벤트가 수집되었습니다.")
    
    return df

# 사용 예시
if __name__ == "__main__":
    # 예시 URL 리스트 (실제 사용 시 본인의 URL 리스트로 교체)
    timeline_urls = urls
    
    results = scrape_timeline_data(timeline_urls)
    print("\n수집된 데이터 미리보기:")
    print(results.head(20))


타임라인 1/446 처리 중: https://gol.gg/game/stats/65431/page-timeline/
  -> 69개의 이벤트 수집
타임라인 2/446 처리 중: https://gol.gg/game/stats/65432/page-timeline/
  -> 49개의 이벤트 수집
타임라인 3/446 처리 중: https://gol.gg/game/stats/65428/page-timeline/
  -> 54개의 이벤트 수집
타임라인 4/446 처리 중: https://gol.gg/game/stats/65429/page-timeline/
  -> 70개의 이벤트 수집
타임라인 5/446 처리 중: https://gol.gg/game/stats/65465/page-timeline/
  -> 57개의 이벤트 수집
타임라인 6/446 처리 중: https://gol.gg/game/stats/65466/page-timeline/
  -> 59개의 이벤트 수집
타임라인 7/446 처리 중: https://gol.gg/game/stats/65467/page-timeline/
  -> 69개의 이벤트 수집
타임라인 8/446 처리 중: https://gol.gg/game/stats/65462/page-timeline/
  -> 68개의 이벤트 수집
타임라인 9/446 처리 중: https://gol.gg/game/stats/65463/page-timeline/
  -> 63개의 이벤트 수집
타임라인 10/446 처리 중: https://gol.gg/game/stats/65471/page-timeline/
  -> 70개의 이벤트 수집
타임라인 11/446 처리 중: https://gol.gg/game/stats/65472/page-timeline/
  -> 57개의 이벤트 수집
타임라인 12/446 처리 중: https://gol.gg/game/stats/65468/page-timeline/
  -> 64개의 이벤트 수집
타임라인 13/446 처리 중: https:/